## Lab 4: Previsão de Eleição de Deputados 

Autora: Lília Sampaio

Neste trabalho queremos prever através do uso de modelos de classificação quais candidatos a deputado foram eleitos em 2014. Primeiro, importamos as bibliotecas necessárias para fazer uma análise descritiva dos dados e conhecer suas propriedades. 

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib

import matplotlib.pyplot as plt
from scipy.stats import skew
from scipy.stats.stats import pearsonr


%config InlineBackend.figure_format = 'retina' #set 'png' here when working on notebook
%matplotlib inline

### 1. Análise descritiva dos dados

Nesta atividade usaremos os dados referentes às eleições para deputado dos anos de 2006 à 2010 para classificar quais candidatos foram eleitos em 2014. Usaremos os dados de 2006 à 2010 como treino, e uma parte dos dados de 2014 como teste. Estes dados são carregados abaixo:

In [2]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

Um trecho inicial desses datasets nos mostra a natureza dos dados e as variáveis que encontramos à disposição para serem usadas na classificação, entre elas estado e partido do candidato, valor total de sua receita, despesas e recursos recebidos:

In [3]:
train.head()

,ano,sequencial_candidato,nome,uf,partido,quantidade_doacoes,quantidade_doadores,total_receita,media_receita,recursos_de_outros_candidatos.comites,...,quantidade_despesas,quantidade_fornecedores,total_despesa,media_despesa,cargo,sexo,grau,estado_civil,ocupacao,situacao
0,2006,10001,JOSÉ LUIZ NOGUEIRA DE SOUSA,AP,PT,6,6,16600.00,2766.67,0.0,...,14,14,16583.60,1184.54,DEPUTADO FEDERAL,MASCULINO,ENSINO MÉDIO COMPLETO,CASADO(A),VEREADOR,nao_eleito
1,2006,10002,LOIVA DE OLIVEIRA,RO,PT,13,13,22826.00,1755.85,6625.0,...,24,23,20325.99,846.92,DEPUTADO FEDERAL,FEMININO,SUPERIOR COMPLETO,SOLTEIRO(A),SERVIDOR PÚBLICO ESTADUAL,nao_eleito
2,2006,10002,MARIA DALVA DE SOUZA FIGUEIREDO,AP,PT,17,16,158120.80,9301.22,2250.0,...,123,108,146011.70,1187.09,DEPUTADO FEDERAL,FEMININO,SUPERIOR COMPLETO,VIÚVO(A),PEDAGOGO,eleito
3,2006,10002,ROMALDO MILANI,MS,PRONA,6,6,3001.12,500.19,0.0,...,8,8,3001.12,375.14,DEPUTADO FEDERAL,MASCULINO,ENSINO MÉDIO INCOMPLETO,CASADO(A),MILITAR REFORMADO,nao_eleito
4,2006,10003,ANSELMO DE JESUS ABREU,RO,PT,48,48,119820.00,2496.25,0.0,...,133,120,116416.64,875.31,DEPUTADO FEDERAL,MASCULINO,ENSINO FUNDAMENTAL COMPLETO,CASADO(A),DEPUTADO,eleito


Uma análise nos valores médios das variáveis, suas medianas, variância e distribuição podem ser vistos abaixo:

In [4]:
train.describe()

,ano,sequencial_candidato,quantidade_doacoes,quantidade_doadores,total_receita,media_receita,recursos_de_outros_candidatos.comites,recursos_de_pessoas_fisicas,recursos_de_pessoas_juridicas,recursos_proprios,recursos_de_partido_politico,quantidade_despesas,quantidade_fornecedores,total_despesa,media_despesa
count,7622.000000,7.622000e+03,7622.000000,7622.000000,7.622000e+03,7622.000000,7.622000e+03,7.622000e+03,7.622000e+03,7.622000e+03,7.622000e+03,7622.000000,7622.000000,7.622000e+03,7622.000000
mean,2008.120703,9.039623e+10,27.673839,25.239963,1.732199e+05,5456.131027,1.965750e+04,2.315820e+04,7.921689e+04,2.577600e+04,2.541136e+04,130.485174,107.849777,1.554507e+05,1346.643975
std,1.996485,9.893847e+10,116.511888,101.804938,4.444187e+05,14003.313610,1.041636e+05,6.689215e+04,2.514138e+05,1.495312e+05,1.308613e+05,411.039689,333.905884,3.901120e+05,6164.552399
min,2006.000000,1.000100e+04,1.000000,1.000000,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000,1.000000,0.000000e+00,0.000000
25%,2006.000000,1.055500e+04,4.000000,3.000000,2.829208e+03,625.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,6.000000,5.000000,2.689600e+03,352.500000
50%,2010.000000,5.000000e+10,9.000000,8.000000,1.334511e+04,1537.775000,1.125000e+03,1.350000e+03,0.000000e+00,1.121600e+03,0.000000e+00,20.000000,18.000000,1.226735e+04,711.365000
75%,2010.000000,1.900000e+11,23.000000,21.000000,1.046682e+05,4666.247500,6.196375e+03,1.574625e+04,1.725875e+04,9.600000e+03,0.000000e+00,80.000000,68.000000,9.633950e+04,1399.812500
max,2010.000000,2.700000e+11,6997.000000,5800.000000,5.690642e+06,500180.000000,3.664205e+06,1.403049e+06,3.178226e+06,5.525600e+06,2.745700e+06,9932.000000,8359.000000,4.989491e+06,500177.000000


Ainda podemos ver que o conjunto de dados conta com $7622$ entradas, mas algumas colunas apresentam valores não informados:

In [5]:
train.count()

ano                                      7622
sequencial_candidato                     7622
nome                                     7622
uf                                       7622
partido                                  7622
quantidade_doacoes                       7622
quantidade_doadores                      7622
total_receita                            7622
media_receita                            7622
recursos_de_outros_candidatos.comites    7622
recursos_de_pessoas_fisicas              7622
recursos_de_pessoas_juridicas            7622
recursos_proprios                        7622
recursos_de_partido_politico             7622
quantidade_despesas                      7622
quantidade_fornecedores                  7622
total_despesa                            7622
media_despesa                            7622
cargo                                    7622
sexo                                     7622
grau                                     7622
estado_civil                      

### 2. Análise de balanceamento das classes

Após conhecer os dados, queremos conhecer as variáveis que apresentam algum tipo de viés e realizar as devidas transformações, preencher valores não informados que sejam de interesse e retirar dados que não ajudem nas nossas predições. 

#### 2.1. Lidando com valores não informados

Inicialmente, checamos se existem dados faltando no conjunto de treino, e verificamos que todas as entradas possuem todos os atributos preenchidos:

In [6]:
train.isnull().sum()

ano                                      0
sequencial_candidato                     0
nome                                     0
uf                                       0
partido                                  0
quantidade_doacoes                       0
quantidade_doadores                      0
total_receita                            0
media_receita                            0
recursos_de_outros_candidatos.comites    0
recursos_de_pessoas_fisicas              0
recursos_de_pessoas_juridicas            0
recursos_proprios                        0
recursos_de_partido_politico             0
quantidade_despesas                      0
quantidade_fornecedores                  0
total_despesa                            0
media_despesa                            0
cargo                                    0
sexo                                     0
grau                                     0
estado_civil                             0
ocupacao                                 0
situacao   

#### 2.2. Balanceamento de classes

Queremos ver o balanceamento entre as classes definidas pela variável ```situacao```, que indica se o candidato foi eleito ou não nos anos de 2006 e 2010. Para isso, vamos ver quantas entradas constam como ```ELEITO``` e ```NAO_ELEITO```:

In [7]:
train["situacao"].value_counts()

nao_eleito    6596
eleito        1026
Name: situacao, dtype: int64

O que em proporção seria:

In [8]:
train["situacao"].value_counts(normalize = True)

nao_eleito    0.86539
eleito        0.13461
Name: situacao, dtype: float64

Ou seja, aproximadamente ```86%``` das entradas constam como não eleitos, e ```13%``` eleitos. Em proporção, para cada candidato marcado como eleito, aproximadamente 7 são nao_eleitos. Claramente vemos que há um desbalanceamento entre as classes. Isso pode causar efeitos colaterais na predição como decisões enviesadas, overfitting, e de maneira mais prática, pode significar que o classificador vai acertar mais para o lado com maior numero de ocorrências, o que no nosso caso seria refletido em um número maior de acertos sobre quem não vai se eleger do que sobre quem de fato seria eleito. 

Para resolver esse problema podemos fazer uso de validação cruzada, utilização de parâmetros que possam ser tunados nos modelos, bem como técnicas de *downsampling*, significando uma diminuição da amostragem da classe majoritária, ou *upsampling*, o aumento da amostragem da classe minoritária.

#### 2.3. Eliminando variáveis

In [9]:
train.drop(['nome'], axis=1, inplace=True)
train.drop(['uf'], axis=1, inplace=True)
train.drop(['partido'], axis=1, inplace=True)
train.drop(['cargo'], axis=1, inplace=True)
train.drop(['grau'], axis=1, inplace=True)
train.drop(['estado_civil'], axis=1, inplace=True)
train.drop(['ocupacao'], axis=1, inplace=True)
train.drop(['sexo'], axis=1, inplace=True)
train.drop(['sequencial_candidato'], axis=1, inplace=True)
train.drop(['ano'], axis=1, inplace=True)

In [32]:
test.drop(['nome'], axis=1, inplace=True)
test.drop(['uf'], axis=1, inplace=True)
test.drop(['partido'], axis=1, inplace=True)
test.drop(['cargo'], axis=1, inplace=True)
test.drop(['grau'], axis=1, inplace=True)
test.drop(['estado_civil'], axis=1, inplace=True)
test.drop(['ocupacao'], axis=1, inplace=True)
test.drop(['sexo'], axis=1, inplace=True)
test.drop(['sequencial_candidato'], axis=1, inplace=True)
test.drop(['ano'], axis=1, inplace=True)

KeyError: "['nome'] not found in axis"

### 3. Treinando modelos

In [21]:
train = pd.get_dummies(train)
X_train = train.loc[:, train.columns != 'situacao_eleito']
y_train = train.situacao_eleito

#### 3.1. Regressão logística



In [30]:
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression(random_state=0, solver='lbfgs')
classifier.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=0, solver='lbfgs',
          tol=0.0001, verbose=0, warm_start=False)

#### 3.2. KNN

In [11]:
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import cross_val_score

from math import sqrt

def calculate_knn(matrix, target):
    rmse_val = [] #to store rmse values for different k
    
    for K in range(100):
        K = K+1
        model = KNeighborsRegressor(n_neighbors = K)
        model.fit(matrix, target)  #fit the model

        errors = rmse_cv(model, matrix, target)
        rmse_val.append(errors.mean()) #store rmse values
    
    print('O menor valor de RMSE é', min(rmse_val), 'para K =', rmse_val.index(min(rmse_val)))
    return rmse_val

def rmse_cv(model, matrix, target):
    rmse = np.sqrt(-cross_val_score(model, matrix, target, scoring="neg_mean_squared_error", cv = 5))
    return(rmse)

In [37]:
calculate_knn(X_train, y_train)

O menor valor de RMSE é 0.24611132177825595 para K = 85


[0.34120820501234805,
 0.2977952651476784,
 0.2752828067916742,
 0.26764003853699914,
 0.2637347194976107,
 0.26251850698876794,
 0.2580119856073579,
 0.2563337275940868,
 0.2555758942470335,
 0.2546236651006812,
 0.2543121348357561,
 0.25332629367133574,
 0.2523890611704444,
 0.2519625286588098,
 0.2514701843970694,
 0.2514663808980394,
 0.2507582113326659,
 0.25040444475595314,
 0.2497817726050488,
 0.24930179499415767,
 0.24886483850206761,
 0.24861845371603727,
 0.2484320768893249,
 0.2484964313031282,
 0.24843334435913006,
 0.24826268508338134,
 0.24854773521772278,
 0.24837918623617808,
 0.24829979867022228,
 0.2483791156401586,
 0.24825249745320419,
 0.24808102079027913,
 0.2479285831057799,
 0.24796253922504657,
 0.24799315470569247,
 0.2480455003725705,
 0.2479551389027576,
 0.248142615835377,
 0.24803385856435808,
 0.24809065668139318,
 0.24811170910976937,
 0.24794789534215292,
 0.24794394455796712,
 0.24773044482926423,
 0.2475269284393943,
 0.24742644742859415,
 0.24721634